In [1]:
#saccades
#three or five digits
#transparsenecy 

In [2]:
%cd /Users/typhaine/Documents/Doc Gorilla/WhereIsMyMNIST/notebooks

/Users/typhaine/Documents/Doc Gorilla/WhereIsMyMNIST/notebooks


In [3]:
import os
import imageio
import random
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import pygame
from psychopy import visual, core, event, gui, data

from what import WhatShift, WhatBackground, WhatNet, WhatTrainer, What, train, test, MNIST, MotionCloudNoise
import MotionClouds as mc

from main import init
args = init(filename='../data/2019-06-12') # pas de drop out!
args


pygame 2.0.1 (SDL 2.0.14, Python 3.7.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


{'w': 28,
 'minibatch_size': 100,
 'train_batch_size': 50000,
 'test_batch_size': 10000,
 'noise_batch_size': 1000,
 'mean': 0.1307,
 'std': 0.3081,
 'what_offset_std': 15,
 'what_offset_max': 25,
 'N_pic': 128,
 'offset_std': 30,
 'offset_max': 34,
 'noise': 0.75,
 'contrast': 0.7,
 'sf_0': 0.1,
 'B_sf': 0.1,
 'do_mask': True,
 'N_theta': 6,
 'N_azimuth': 24,
 'N_eccentricity': 10,
 'N_phase': 2,
 'rho': 1.41,
 'bias_deconv': True,
 'p_dropout': 0.0,
 'dim1': 1000,
 'dim2': 1000,
 'lr': 0.005,
 'do_adam': True,
 'bn1_bn_momentum': 0.5,
 'bn2_bn_momentum': 0.5,
 'momentum': 0.3,
 'epochs': 60,
 'num_processes': 1,
 'no_cuda': False,
 'log_interval': 100,
 'verbose': 1,
 'filename': '../data/2019-06-12',
 'seed': 2019,
 'N_cv': 10,
 'do_compute': True,
 'save_model': True}

In [4]:
## Unit tests

#transforms.GaussianBlur(kernel_size = 1, sigma=(0.1, 2.0))
transform=transforms.Compose([
                               WhatShift(args,i_offset=0, j_offset=0),
                               WhatBackground(contrast = args.contrast,
                                              noise=0, 
                                              sf_0=args.sf_0, 
                                              B_sf=args.B_sf),
                               transforms.ToTensor(),
                               transforms.Normalize((args.mean,), (args.std,))
                           ])

dataset_train = MNIST('../data',
                        train=True,
                        download=True,
                        transform=transform,
                        )

train_loader = torch.utils.data.DataLoader(dataset_train,
                                         batch_size=args.minibatch_size,
                                         shuffle=True)

dataset_test = MNIST('../data',
                        train=False,
                        download=True,
                        transform=transform,
                        )

test_loader = torch.utils.data.DataLoader(dataset_test,
                                         batch_size=args.minibatch_size,
                                         shuffle=True)

data, label = next(iter(train_loader))
#%timeit data, label = next(iter(train_loader))
print(label.shape)
print(data.shape)

torch.Size([100])
torch.Size([100, 1, 128, 128])


In [5]:
def add(data):
    data = data.numpy().reshape(100,128,128)
    data = np.array([[np.max((random_roll(data[random.randint(0,99)],0,0),
                random_roll(data[random.randint(0,99)],50,50),
                random_roll(data[random.randint(0,99)],-50,-50),
                random_roll(data[random.randint(0,99)],-50,50),
                random_roll(data[random.randint(0,99)],50,-50)),axis = 0)]
            for i in range(data.shape[0])]).reshape(100,128,128)
    return data

In [6]:
def RGBA(data):
    data = np.interp(data, (data.min(), data.max()), (0, 1))
    alpha = np.ones((data.shape[0],128,128))
    return np.stack((data,data,data,alpha),axis=-1)

In [7]:
def random_roll(data, x_shift, y_shift):
    return np.roll(np.roll(data, x_shift, axis = 0), y_shift, axis = 1)

In [8]:
def motion(data,background):
    data = np.interp(data, (data.min(), data.max()), (0, 1))
    background = RGBA(background)
    return np.array([[np.max((background[back,:,:,:],data[D,:,:,:]),axis=0) 
                      for back in range(background.shape[0])] 
                     for D in range(data.shape[0])])
            

In [9]:
z, env = MotionCloudNoise(sf_0=args.sf_0, B_sf=args.B_sf, alpha=.0, N_pic=128, seed=42, motion = True)
data = motion(RGBA(add(data)),np.moveaxis(z,(0,1,2),(2,1,0)))
data.shape

(100, 1, 128, 128, 4)

In [10]:
instructions = """
At each trial you see a series of textured images, report if you 
spot a specific digit by pressing the space bar.

Press the space bar to begin the experiment.

"""

win = visual.Window(winType = 'pyglet', size =(800,800), colorSpace = "rgb")

message = visual.TextStim(win, text= instructions)
message.autoDraw = True  # Automatically draw every frame
win.flip()
core.wait(5.0)
message.text = ''  # Change properties of existing stim
win.flip()
core.wait(0.0)

while True:
    for i in range(data.shape[0]-1):
            mov = data[i]
            for ii in range(mov.shape[0]):
                movie_clip = visual.ImageStim(win, mov[ii], colorSpace = "rgb", contrast = 1, size = 2, flipVert=True)
                movie_clip.draw()
                win.flip()

KeyboardInterrupt: 

In [11]:
def overlay(data):
    overlay_data = np.empty((128,128))
    for i in range(len(data)-1):
        random_number = random.randint(0,9)
        if random_number == 0:
            overlay_data = np.vstack((overlay_data,random_roll(data)))
        elif random_number == 1:
            number = np.max((random_roll(data),random_roll(data)), axis=0)
            overlay_data = np.vstack((overlay_data,number))
        elif random_number == 2:
            number = np.max((random_roll(data),random_roll(data),random_roll(data)), axis=0)
            overlay_data = np.vstack((overlay_data,number))
        elif random_number == 3:
            number = np.max((random_roll(data),random_roll(data),random_roll(data),random_roll(data)), axis=0)
            overlay_data = np.vstack((overlay_data,number))
        elif random_number == 4:
            number = np.max((random_roll(data),random_roll(data),random_roll(data),random_roll(data),
                            random_roll(data)), axis=0)
            overlay_data = np.vstack((overlay_data,number))
        elif random_number == 5:
            number = np.max((random_roll(data),random_roll(data),random_roll(data),random_roll(data),
                            random_roll(data),random_roll(data)), axis=0)
            overlay_data = np.vstack((overlay_data,number))
        elif random_number == 6:
            number = np.max((random_roll(data),random_roll(data),random_roll(data),random_roll(data),
                            random_roll(data),random_roll(data),random_roll(data)), axis=0)
            overlay_data = np.vstack((overlay_data,number))
        elif random_number == 7:
            number = np.max((random_roll(data),random_roll(data),random_roll(data),random_roll(data),
                            random_roll(data),random_roll(data),random_roll(data),random_roll(data)), axis=0)
            overlay_data = np.vstack((overlay_data,number))
        elif random_number == 8:
            number = np.max((random_roll(data),random_roll(data),random_roll(data),random_roll(data),
                            random_roll(data),random_roll(data),random_roll(data),random_roll(data),
                            random_roll(data)), axis=0)
            overlay_data = np.vstack((overlay_data,number))
        elif random_number == 9:
            number = np.max((random_roll(data),random_roll(data),random_roll(data),random_roll(data),
                            random_roll(data),random_roll(data),random_roll(data),random_roll(data),
                            random_roll(data),random_roll(data)), axis=0)
            overlay_data = np.vstack((overlay_data,number))
    overlay_data = overlay_data.reshape(100,128,128)
    return overlay_data

In [13]:
def combine(data,background):
    combine_MNIST = np.empty((128,128))
    ran_gen = random.randint(0,99)
    MNIST = data[ran_gen]
    MNIST = np.interp(MNIST, (MNIST.min(), MNIST.max()), (-1, 1))
    for i in range(len(background)-1):
        back = np.interp(background[i], (background[i].min(), background[i].max()), (-1, +1))
        fused = np.max((back,MNIST), axis=0)
        combine_MNIST = np.vstack((combine_MNIST,fused))
    combine_MNIST = combine_MNIST.reshape(60,128,128)
    return combine_MNIST

In [14]:
def multiple(data,moving_background):
    big = combine(data,moving_background)
    for i in range(0,99,1):
        temp = combine(data,moving_background)
        big = np.vstack((big,temp))
    return big.reshape(6000,128,128)

In [15]:
background_int = background
expanded_MNIST1 = expand_MNIST(data)
expanded_MNIST2 = overlay(expanded_MNIST1)
test = multiple(expanded_MNIST2,background_int)